In [12]:
import pandas as pd
import numpy as np

data = pd.read_csv(r"data/Seasons_Stats.csv")
data.drop(columns=['Unnamed: 0'],inplace = True)
data.head(10)

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0
5,1950.0,Ed Bartels,F,24.0,NYK,2.0,NaN,NaN,NaN,0.376,...,0.667,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2.0,4.0
6,1950.0,Ralph Beard,G,22.0,INO,60.0,NaN,NaN,NaN,0.422,...,0.762,NaN,NaN,NaN,233.0,NaN,NaN,NaN,132.0,895.0
7,1950.0,Gene Berce,G-F,23.0,TRI,3.0,NaN,NaN,NaN,0.275,...,0.000,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.0,10.0
8,1950.0,Charlie Black,F-C,28.0,TOT,65.0,NaN,NaN,NaN,0.346,...,0.651,NaN,NaN,NaN,163.0,NaN,NaN,NaN,273.0,661.0
9,1950.0,Charlie Black,F-C,28.0,FTW,36.0,NaN,NaN,NaN,0.362,...,0.632,NaN,NaN,NaN,75.0,NaN,NaN,NaN,140.0,382.0


In [26]:
data_2016 = data.query("Year == 2016")
data_2017 = data.query("Year == 2017")
data_2017.head(10)
data_2017.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 595 entries, 24096 to 24690
Data columns (total 52 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    595 non-null    float64
 1   Player  595 non-null    object 
 2   Pos     595 non-null    object 
 3   Age     595 non-null    float64
 4   Tm      595 non-null    object 
 5   G       595 non-null    float64
 6   GS      595 non-null    float64
 7   MP      595 non-null    float64
 8   PER     595 non-null    float64
 9   TS%     593 non-null    float64
 10  3PAr    593 non-null    float64
 11  FTr     593 non-null    float64
 12  ORB%    595 non-null    float64
 13  DRB%    595 non-null    float64
 14  TRB%    595 non-null    float64
 15  AST%    595 non-null    float64
 16  STL%    595 non-null    float64
 17  BLK%    595 non-null    float64
 18  TOV%    593 non-null    float64
 19  USG%    595 non-null    float64
 20  blanl   0 non-null      float64
 21  OWS     595 non-null    float64
 

In [25]:
data_2016 = data.query("Year == 2016")
data_2016.loc[data_2016['Player'] == 'Stephen Curry','MVP'] = 1
data_2016['MVP'].fillna(0, inplace=True)
data_2016.sort_values(by=["MVP"],ascending=False,inplace=True)
#data_2016.query("Player == 'Stephen Curry'")
data_2016

c:\Users\THINK\anaconda3\envs\DataScience\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\Users\THINK\anaconda3\envs\DataScience\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
c:\Users\THINK\anaconda3\envs\DataScience\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,MVP
23633,2016.0,Stephen Curry,PG,27.0,GSW,79.0,79.0,2700.0,31.5,0.669,...,68.0,362.0,430.0,527.0,169.0,15.0,262.0,161.0,2375.0,1.0
23517,2016.0,Quincy Acy,PF,25.0,SAC,59.0,29.0,876.0,14.7,0.629,...,65.0,123.0,188.0,27.0,29.0,24.0,27.0,103.0,307.0,0.0
23906,2016.0,Emmanuel Mudiay,PG,19.0,DEN,68.0,66.0,2068.0,9.9,0.437,...,32.0,200.0,232.0,372.0,65.0,35.0,217.0,141.0,872.0,0.0
23900,2016.0,Markieff Morris,PF,26.0,TOT,64.0,45.0,1629.0,12.2,0.489,...,73.0,279.0,352.0,124.0,58.0,34.0,135.0,184.0,765.0,0.0
23901,2016.0,Markieff Morris,PF,26.0,PHO,37.0,24.0,917.0,10.8,0.456,...,43.0,151.0,194.0,87.0,33.0,17.0,90.0,104.0,430.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23711,2016.0,Jeff Green,SF,29.0,TOT,80.0,41.0,2253.0,13.0,0.510,...,74.0,258.0,332.0,138.0,58.0,40.0,93.0,164.0,939.0,0.0
23712,2016.0,Jeff Green,SF,29.0,MEM,53.0,31.0,1544.0,13.4,0.517,...,55.0,186.0,241.0,98.0,40.0,19.0,69.0,102.0,646.0,0.0
23713,2016.0,Jeff Green,SF,29.0,LAC,27.0,10.0,709.0,12.0,0.496,...,19.0,72.0,91.0,40.0,18.0,21.0,24.0,62.0,293.0,0.0
23714,2016.0,Blake Griffin,PF,26.0,LAC,35.0,35.0,1170.0,22.1,0.544,...,51.0,243.0,294.0,170.0,28.0,17.0,83.0,95.0,749.0,0.0
